In [ ]:
import sys 
#path to append: "./ChimpRec/Code"
sys.path.append(...)

from chimplib.imports import plt, os, np, Image
from chimplib.utils import yolo_to_pixel_coord, face_to_yolo_relative_to_body

In [ ]:
#path to facial recognition dataset folder
chimprec_recognition_dataset_path = ...
#path of the folder in which the dataset is to be stored
output_dataset = ...

In [ ]:
counter = 0
for set in ["train", "val"]:
    #dirs contains a list of folder names of each individual
    dirs = []
    for entry in os.listdir(os.path.join(chimprec_recognition_dataset_path, set)):
        entry_path = os.path.join(chimprec_recognition_dataset_path, entry)
        if os.path.isdir(entry_path):
            dirs.append(entry)

    for dir in dirs:
        images_dir = os.path.join(chimprec_recognition_dataset_path, set, dir, "images")
        labels_dir = os.path.join(chimprec_recognition_dataset_path, set, dir, "labels")

        #recursively browse the folder and its subfolders
        for _, _, files in os.walk(images_dir):
            for file in files:
                img_path = os.path.join(images_dir, file)
                label_path = os.path.join(labels_dir, file.replace(".png", ".txt"))

                image = Image.open(img_path)
                frame = np.array(image)

                face_coord = None
                body_coord = None

                with open(label_path, "r") as f:
                    for line in f:
                        values = line.strip().split(" ")
                        
                        #1 is the id assigned to body bbox annotations and 0 to face bbox annotations
                        if values[0] == "1":
                            body_coord = tuple(map(float, values[1:]))
                        elif values[0] == "0":
                            face_coord = tuple(map(float, values[1:]))

                if body_coord == None or face_coord == None: 
                    continue
                
                #crop the image around the body then save in the right place
                height, width, _ = frame.shape
                bx1, by1, bx2, by2 = yolo_to_pixel_coord(body_coord, width, height)
                cropped_body = frame[by1:by2, bx1:bx2]
                image_pil = Image.fromarray(np.uint8(cropped_body))
                image_pil.save(f"{output_dataset}/images/chimp{counter}.png")

                #recalculate the face bbox according to the iamge crop around the body and save in the right place
                fx1, fy1, fx2, fy2 = yolo_to_pixel_coord(face_coord, width, height)
                relative_face = face_to_yolo_relative_to_body((bx1, by1, bx2, by2), (fx1, fy1, fx2, fy2))
                with open(f"{output_dataset}/labels/chimp{counter}.txt", "w") as file:
                    file.write("0 " + " ".join(map(str, relative_face))) 
                
                counter += 1

AD
BL
BS
DK
IV
JJ
KG
KM
KR
LM
MG
MK
MM
MZ
NJ
NR
PD
TC
TS
TT
